In [19]:
import tensorflow as tf
from tensorflow.keras import layers
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [20]:
# Data Collection Functions (keep yours)
def collect_image_path(root_dir):
    data = []
    classes = []    
    for folder_classes in os.listdir(root_dir):
        class_path = os.path.join(root_dir,folder_classes)
        classes.append(folder_classes)        
        if os.path.isdir(class_path):
            for split in ['Train' , 'Test']:
                split_path = os.path.join(class_path,split)
                if os.path.isdir(split_path):
                    for image_path in os.listdir(split_path):
                        image_path = os.path.join(split_path , image_path)
                        data.append({
                            'path':image_path,
                            'target': folder_classes,
                            'split':split
                        })
    return data, classes

In [21]:
# Dataset Configuration
root_dir = 'Bone Break Classification/'
data, classes = collect_image_path(root_dir)
batch_size = 32
IMG_SIZE = (224, 224)

# Split dataset
train_data = [entry for entry in data if entry['split'] == 'Train']
test_data = [entry for entry in data if entry['split'] == 'Test']

# Prepare paths and labels
train_paths = [entry['path'] for entry in train_data]
train_labels = [entry['target'] for entry in train_data]
test_paths = [entry['path'] for entry in test_data]
test_labels = [entry['target'] for entry in test_data]

# Encode labels
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

In [22]:
# Dataset Configuration
root_dir = 'Bone Break Classification/'
data, classes = collect_image_path(root_dir)
batch_size = 32
IMG_SIZE = (224, 224)

# Split dataset
train_data = [entry for entry in data if entry['split'] == 'Train']
test_data = [entry for entry in data if entry['split'] == 'Test']

# Prepare paths and labels
train_paths = [entry['path'] for entry in train_data]
train_labels = [entry['target'] for entry in train_data]
test_paths = [entry['path'] for entry in test_data]
test_labels = [entry['target'] for entry in test_data]

# Encode labels
le = LabelEncoder()
train_labels = le.fit_transform(train_labels)
test_labels = le.transform(test_labels)

In [23]:
# TF Dataset Pipeline
def process_path(image_path, label):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=1)  # Read as grayscale
    img = tf.image.grayscale_to_rgb(img)  # Convert to 3 channels
    img = tf.image.resize(img, IMG_SIZE)
    return img, label

In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_paths, train_labels))
train_dataset = train_dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_paths, test_labels))
test_dataset = test_dataset.map(process_path, num_parallel_calls=tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1)
])

In [25]:
# Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1)
])

In [26]:
# Transfer Learning Model with Fixed Preprocessing
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False  # Freeze base model

# Corrected Model Architecture
model = tf.keras.Sequential([
    layers.Input(shape=(224, 224, 3)),
    data_augmentation,
    layers.Lambda(tf.keras.applications.mobilenet_v2.preprocess_input),  # Wrap in Lambda layer
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.5),
    layers.Dense(256, activation='relu'),
    layers.Dense(len(classes), activation='softmax')
])


In [27]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Train the model
epochs = 15
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=epochs
)

Epoch 1/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 13s 264ms/step - accuracy: 0.1295 - loss: 2.7699 - val_accuracy: 0.2143 - val_loss: 2.1976
Epoch 2/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 231ms/step - accuracy: 0.2322 - loss: 2.1832 - val_accuracy: 0.1786 - val_loss: 2.1662
Epoch 3/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 8s 233ms/step - accuracy: 0.2601 - loss: 2.0825 - val_accuracy: 0.2357 - val_loss: 2.1408
Epoch 4/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 230ms/step - accuracy: 0.2815 - loss: 2.0682 - val_accuracy: 0.2286 - val_loss: 2.1418
Epoch 5/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 225ms/step - accuracy: 0.3082 - loss: 2.0251 - val_accuracy: 0.2143 - val_loss: 2.1662
Epoch 6/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 224ms/step - accuracy: 0.3241 - loss: 1.9454 - val_accuracy: 0.2143 - val_loss: 2.1714
Epoch 7/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 223ms/step - accuracy: 0.3167 - loss: 1.9912 - val_accuracy: 0.2286 - val_loss: 2.1256
Epoch 8/15
31/31 ━━━━━━━━━━━━━━━━━━━━ 7s 227ms/step - accuracy: 0.3345 - loss: 1.8867 - val_accuracy: 0